In [ ]:
!pip install pyautogen autogen google-generativeai --quiet

In [ ]:
import google.generativeai as genai
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager
import os

In [ ]:
os.environ["GOOGLE_API_KEY"] = "Your_key"
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

In [ ]:
class StateFlow:
    def __init__(self):
        self.investment_type = None

    def determine_strategy(self, portfolio_data):
        """Determine Growth vs Value based on portfolio"""
        total_investment = portfolio_data['fd'] + portfolio_data['sips'] + portfolio_data['real_estate']
        salary = portfolio_data['salary']

        if total_investment < salary * 2:
            self.investment_type = "GROWTH"
        else:
            self.investment_type = "VALUE"

        return self.investment_type

In [ ]:
llm_config = {
    "config_list": [
        {
            "model": "gemini-1.5-flash",
            "api_key": os.environ["GOOGLE_API_KEY"],
            "api_type": "google"
        }
    ]
}

In [ ]:
# Portfolio Analysis Agent
portfolio_agent = AssistantAgent(
    name="Portfolio_Analysis_Agent",
    llm_config=llm_config,
    system_message="""You are the Portfolio Analysis Agent.
    Analyze the user's investment portfolio and determine if they should pursue Growth or Value investments.

    After receiving portfolio data, provide:
    1. Portfolio summary
    2. Investment category recommendation (Growth or Value)
    3. Brief reasoning for the recommendation"""
)

# Growth Investment Agent
growth_agent = AssistantAgent(
    name="Growth_Investment_Agent",
    llm_config=llm_config,
    system_message="""You are the Growth Investment Agent.
    Provide high-growth investment recommendations:
    - Equity mutual funds
    - Growth stocks
    - High-risk high-reward options

    Give specific investment suggestions with expected returns."""
)

# Value Investment Agent
value_agent = AssistantAgent(
    name="Value_Investment_Agent",
    llm_config=llm_config,
    system_message="""You are the Value Investment Agent.
    Provide stable, long-term investment recommendations:
    - Debt funds
    - Fixed deposits
    - Dividend stocks
    - Conservative options

    Give specific investment suggestions focusing on stability."""
)

# Investment Advisor Agent
advisor_agent = AssistantAgent(
    name="Investment_Advisor_Agent",
    llm_config=llm_config,
    system_message="""You are the Investment Advisor Agent.
    Create a personalized financial report based on:
    - Portfolio analysis
    - Investment recommendations from Growth/Value agents

    Provide a comprehensive final report with actionable recommendations."""
)

# User Proxy Agent
user_proxy = UserProxyAgent(
    name="User_Proxy_Agent",
    human_input_mode="NEVER",
    system_message="You represent the user and initiate the portfolio management process."
)


In [ ]:
def run_portfolio_manager():
    # Step 1: Collect user inputs
    print("=== Financial Portfolio Manager ===")
    salary = float(input("Current salary: "))
    fd = float(input("Fixed Deposits amount: "))
    sips = float(input("SIPs amount: "))
    real_estate = float(input("Real Estate value: "))

    portfolio_data = {
        'salary': salary,
        'fd': fd,
        'sips': sips,
        'real_estate': real_estate
    }

    # Step 2: StateFlow determines strategy
    state_flow = StateFlow()
    strategy = state_flow.determine_strategy(portfolio_data)
    print(f"\n✅ StateFlow Decision: {strategy} Investment Strategy")

    # Step 3: Select appropriate agents based on StateFlow
    if strategy == "GROWTH":
        investment_agent = growth_agent
    else:
        investment_agent = value_agent

    # Step 4: Create Group Chat
    group_chat = GroupChat(
        agents=[user_proxy, portfolio_agent, investment_agent, advisor_agent],
        messages=[],
        max_round=8
    )

    group_chat_manager = GroupChatManager(
        groupchat=group_chat,
        llm_config=llm_config
    )

    # Step 5: Start the conversation
    initial_message = f"""
    Portfolio Management Request:

    User Portfolio Data:
    - Salary: ₹{salary:,.2f}
    - Fixed Deposits: ₹{fd:,.2f}
    - SIPs: ₹{sips:,.2f}
    - Real Estate: ₹{real_estate:,.2f}

    StateFlow has determined: {strategy} Investment Strategy

    Portfolio_Analysis_Agent: Please analyze this portfolio and provide insights.
    {investment_agent.name}: Please provide {strategy.lower()} investment recommendations.
    Investment_Advisor_Agent: Please compile the final personalized financial report.
    """

    # Step 6: Execute Group Chat
    print("\n🚀 Starting Group Chat...")

    result = user_proxy.initiate_chat(
        recipient=group_chat_manager,
        message=initial_message,
        max_turns=6
    )

    print(f"\n✅ Portfolio Management Complete!")
    print(f"📊 Strategy Used: {strategy}")
    print(f"💼 Total Portfolio: ₹{fd + sips + real_estate:,.2f}")

In [ ]:
if __name__ == "__main__":
    run_portfolio_manager()